Dr Oliviero Andreussi, olivieroandreuss@boisestate.edu

Boise State University, Department of Chemistry and Biochemistry

# Fitting and Data Analysis of Vibrational-Rotational Spectroscopy of HCl and DCl {-}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression

In [ ]:
# I am assuming the file in question will be located in a `RoVib_Data/` subfolder in your `Colab Notebook/` folder. 
# I will be using files named `H35Cl_peaks.csv`,`H37Cl_peaks.csv`,`D35Cl_peaks.csv`,`D37Cl_peaks.csv`
# Make sure to change these names if you use different files.
# Load the google drive with your files 
from google.colab import drive
drive.mount('/content/drive')
# The following needs to be the path of the folder with all your datafile in .csv format
path='/content/drive/MyDrive/Colab Notebooks/ParticleBox_Data/'

## Scipy to Find Peaks {-}

Let us start by loading the spectra for one of the two experiments that were performed. We will consider the $HCl$ system, whose raw IR absorption spectrum is saved in the file HCl_0.5cm_4scans.csv. By now you should be familiar on the options of the `Pandas.read_csv()` function.

In [ ]:
file='HCl_0.5cm_4scans.csv'
HCl_spectra=pd.read_csv(path+file,header=3,names=['frequency','absorbance'])

It is always a good idea to print out the data that you just read to make sure that the columns and rows are what you expect. 

In [ ]:
HCl_spectra

It also does not harm to use `DataFrame.info()` to check that the data have the right format. 

In [ ]:
print(HCl_spectra.info())

And it is always a good idea to visualize the data. You can adjust the range of the x and y axis using the `xlim()` and `ylim()` functions of `Matplotlib.Pyplot`. 

In [ ]:
plt.plot(HCl_spectra['frequency'],HCl_spectra['absorbance'])
plt.xlim(2550,3150)

Using our favourite filtering function (i.e. the `DataFrame.query()` method) we can create a new `DataFrame` that only contains the spectra within the frequency range identified above. 

In [ ]:
filtered_spectrum=HCl_spectra.query('frequency > 2550 and frequency < 3150').copy()

We realize that the baseline of the spectrum is not flat, but in the considered range we can think of it as being just a linear drift. We can compute the equation of the line by just taking the endpoints of the spectrum. We can then remove the baseline from the absorbance values. 

In [ ]:
y0 = filtered_spectrum.iloc[0,1]
yf = filtered_spectrum.iloc[-1,1]
deltay = yf - y0
x0 = filtered_spectrum.iloc[0,0]
xf = filtered_spectrum.iloc[-1,0]
deltax = xf - x0
slope = deltay / deltax 
filtered_spectrum['absorbance_shifted']=filtered_spectrum['absorbance']-y0-slope*(filtered_spectrum['frequency']-x0)

Now, using `Scipy.Signal.find_peaks` we can identify the peaks in the spectrum in one go. Note that this function finds maxima, so we need to feed the negative of the absorbance spectrum to it. We can also specify what we consider the height of the peaks. In the example below we set this height to 0.8, as smaller values would find spurious peaks. 

In [ ]:
from scipy.signal import find_peaks
peaks, _ = find_peaks(-1*filtered_spectrum['absorbance_shifted'].values, height=0.8)
plt.plot(filtered_spectrum['frequency'],filtered_spectrum['absorbance_shifted'])
plt.plot(filtered_spectrum.iloc[peaks,0],filtered_spectrum.iloc[peaks,2],'+r')

The peaks above correspond to two different molecules, $H^{35}Cl$ and $H^{37}Cl$. Lukily the peaks are regularly alternating between the two molecules, so we can use slicing of `Numpy` arrays to only select the even/odd peaks. In particular, by looking at the plot above we can see that the even peaks are the most intense peaks, thus they correspond to the $^{37}Cl$ isotope. We can select the even peaks by specifying `[0::2]`, which means all the values in the array starting from index 0, with a step of 2. Similarly, we can select the odd peaks by specifying `[1::2]`, which means all the values in the array starting from index 1, with a step of 2. 

In [ ]:
H37Cl_peaks_scipy=filtered_spectrum.iloc[peaks,0].values[0::2] 
H35Cl_peaks_scipy=filtered_spectrum.iloc[peaks,0].values[1::2]
H37Cl_peaks_scipy.sort() # this will reorder the peaks from smallest to largest
H35Cl_peaks_scipy.sort() # this will reorder the peaks from smallest to largest
print(H37Cl_peaks_scipy)
print(H35Cl_peaks_scipy)

We can read the csv file that contains the peak positions determined using Igor Pro and save it into a `Pandas.DataFrame`.

In [ ]:
H35Cl_peaks=pd.read_csv(path+'H35Cl_peaks.csv')
print(H35Cl_peaks)

We can add a new column to the `DataFrame` with the peaks found using `Scipy`. This way, we can check how different the two fitting procedures are and compare the results. 

In [ ]:
H35Cl_peaks['freq_py']=H35Cl_peaks_scipy
H35Cl_peaks['deltafreq']=H35Cl_peaks['freq_py']-H35Cl_peaks['freq']
print(H35Cl_peaks)

## Printing Pandas Dataframes as Tables {-}

There are different options to convert a `Pandas.DataFrame` into a nice-looking table. One immediate option is to use the `df.to_markdown()` function to create a Markdown version of the table, that we can cut and paste inside a Markdown cell.

In [ ]:
print(H35Cl_peaks.to_markdown(index=False)) # the optional argument allows to remove the index column

This is the table obtained above. We can just cut and past it here to show it in a nice form. NOTE that we can also asjust the Markdown by hand, say to fix the header and add the units

|   m |   Frequency Igor ($cm^{-1}$) | Frequency Python ($cm^{-1}$) |   Error ($cm^{-1}$) |
|----:|--------:|----------:|------------:|
| -11 | 2626    |   2626    |     0.0033  |
| -10 | 2652.23 |   2652.25 |     0.0159  |
|  -9 | 2678    |   2678    |    -0.00289 |
|  -8 | 2703.28 |   2703.25 |    -0.03417 |
|  -7 | 2728.06 |   2728    |    -0.05844 |
|  -6 | 2752.32 |   2752.38 |     0.05773 |
|  -5 | 2776.04 |   2776    |    -0.04444 |
|  -4 | 2799.23 |   2799.25 |     0.02072 |
|  -3 | 2821.86 |   2821.88 |     0.01517 |
|  -2 | 2843.92 |   2843.88 |    -0.04503 |
|  -1 | 2865.39 |   2865.38 |    -0.01707 |
|   1 | 2906.55 |   2906.5  |    -0.05366 |
|   2 | 2926.2  |   2926.25 |     0.04882 |
|   3 | 2945.22 |   2945.25 |     0.03117 |
|   4 | 2963.59 |   2963.62 |     0.03411 |
|   5 | 2981.31 |   2981.25 |    -0.05501 |
|   6 | 2998.35 |   2998.38 |     0.02381 |
|   7 | 3014.72 |   3014.75 |     0.03262 |
|   8 | 3030.39 |   3030.38 |    -0.01807 |
|   9 | 3045.37 |   3045.38 |     0.00871 |
|  10 | 3059.62 |   3059.62 |     0.00082 |
|  11 | 3073.16 |   3073.12 |    -0.0357  |

Or we can use the `tabulate` module to convert the dataframe into an oject that can be printed

In [ ]:
from tabulate import tabulate
print(tabulate(H35Cl_peaks,headers='keys',tablefmt='fancy_grid',showindex=False))

## Polynomial Fit {-}

In order to obtain important molecular properties, we need to be able to fit the positions of the peaks as a function of the quantum number $m$. However, the expression that we need to use is (see Figure 3 and Eq. 10 from the handout):

$\tilde{\nu}(m)=\tilde{\nu}_0+(2B_e-2\alpha _e)m-\alpha _em^2 -4D_em^3$

which involves a cubic dependance on the variable $m$. We can still use linear regression to fit this formula, as the model is still linear in the parameters of the fit (i.e. it is linear in the coefficients of the polynomial function). However, the independent variables that we use in the linear regression need to include the appropriate powers of $m$. We can generate these polynomial features either by hand or automatically using `SKLearn`.

Since we only have one variable, it is relatively straightforward to generate its second and third powers.

In [ ]:
H35Cl_peaks['m2']=H35Cl_peaks['m']**2
H35Cl_peaks['m3']=H35Cl_peaks['m']**3
H35Cl_peaks

We can now use $m$, $m^2$, and $m^3$ as our three separate independent variables for a linear regression fit

In [ ]:
X=H35Cl_peaks[['m','m2','m3']].values
y=H35Cl_peaks['freq']
H35Cl_lr=LinearRegression()
H35Cl_lr.fit(X,y)
H35Cl_lr.score(X,y)
plt.scatter(H35Cl_peaks['m'],H35Cl_peaks['freq'],label='Experiment')
plt.scatter(H35Cl_peaks['m'],H35Cl_lr.predict(X),s=5,label='Fit')
plt.xlabel('m')
plt.ylabel('Frequency ($cm^-1$)')
plt.legend()

The intercept of the fit above will correspond to the fundamental vibrational frequency, $\tilde{\nu}_0$. You can find the value of the intercept in the `LinearRegression.intercept_` attribute, but only after you have performed the fit on your datapoints. 

In [ ]:
H35Cl_lr.intercept_

The linear and quadratic coefficients of the fit depend on $\alpha _e$ and $B_e$, so you will need to do a bit of algebra to obtain the two separate quantities. The cubic coefficient will be directly related to the $D_e$ parameter. You can access these coefficients in the `LinearRegression.coef_` attribute. Note that this attribute will be an array with one entry for each feature used in the model. In our case, there will be three coefficients:


In [ ]:
H35Cl_lr.coef_

Using the formula above, we can get the centrifugal distrortion constant $D_e$ from the last coefficient as:

In [ ]:
De=-H35Cl_lr.coef_[2]/4
print("The De parameter for HCl is {:.2e} cm^-1".format(De))

We can also get the rovibrational coupling constant $\alpha _e$ from the second coefficient

In [ ]:
alpha_e=-H35Cl_lr.coef_[1]
print("The alpha_e parameter for HCl is {:.3f} cm^-1".format(alpha_e))

Eventually, we can get the rotational constant $B_e$ from the first and second coefficients

In [ ]:
Be=H35Cl_lr.coef_[0]/2-H35Cl_lr.coef_[1]
print("The Be parameter for HCl is {:.3f} cm^-1".format(Be))

We can now repeat the analysis above using the peak frequencies obtained with Python, instead of the peaks fitted by Igor Pro. This allows us to indirectly assess the impact of our fit procedure on the calculated molecular properties. From the results below it is clear that only the smallest of the three parameters has some minor discrepancy due to the different fits. 

In [ ]:
X=H35Cl_peaks[['m','m2','m3']].values
y=H35Cl_peaks['freq_py']
H35Cl_lr=LinearRegression()
H35Cl_lr.fit(X,y)
De=-H35Cl_lr.coef_[2]/4
print("The De parameter for HCl is {:.2e} cm^-1".format(De))
alpha_e=-H35Cl_lr.coef_[1]
print("The alpha_e parameter for HCl is {:.3f} cm^-1".format(alpha_e))
Be=H35Cl_lr.coef_[0]/2-H35Cl_lr.coef_[1]
print("The Be parameter for HCl is {:.3f} cm^-1".format(Be))